In [23]:
import cv2
import numpy as np
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [49]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [11]:
img = cv2.imread("4.jpg")

In [18]:
cv2.imshow('Original Image', img)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [20]:
h,w = img.shape[0:2]

In [36]:
T = [4, 10, 15, 25, 40, 90, 125, 160, 200]

In [22]:

for index in T:
    img = cv2.imread(f"{index}.jpg")
    cropped = img[0:h, 0:w]
    cv2.imwrite(f"{index}-p.jpg", cropped)
    

In [37]:
channels = {}
cB = []
cG = []
cR = []
for index in T:
    img = cv2.imread(f"{index}-p.jpg")
    b, g, r = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    b = np.mean(b)
    g = np.mean(g)
    r = np.mean(r)
    channels[index] = (b, g, r)
    cB.append(b)
    cG.append(g)
    cR.append(r)
    print(f"{index}-p.jpg r:{r}, g:{g}, b:{b}")

4-p.jpg r:238.05122085048012, g:222.7633196159122, b:204.29733882030177
10-p.jpg r:201.50075445816188, g:185.0356241426612, b:169.11200274348423
15-p.jpg r:167.94796982167352, g:155.29267489711935, b:139.71318244170095
25-p.jpg r:133.68124828532237, g:123.83524005486969, b:111.73971193415638
40-p.jpg r:99.42213991769547, g:92.35936899862826, b:81.27004115226337
90-p.jpg r:59.75310013717421, g:53.705884773662554, b:47.52319615912209
125-p.jpg r:41.337475994513035, g:36.461001371742114, b:30.440438957475994
160-p.jpg r:34.569533607681755, g:30.392894375857338, b:25.679355281207133
200-p.jpg r:15.265020576131688, g:14.543319615912209, b:10.404266117969822


In [38]:
T.reverse()
cB.reverse()
cG.reverse()
cR.reverse()

In [44]:
Ts = [1/t for t in T]
    

In [51]:
sns.lineplot(x = Ts, y = cB, marker='o')

In [45]:
Ts

[0.005,
 0.00625,
 0.008,
 0.011111111111111112,
 0.025,
 0.04,
 0.06666666666666667,
 0.1,
 0.25]